This section will explain how to extract the locations of trees from aerial imagery and create a shapefile containing points representing these tree locations. 

importing neccesery libraries

In [1]:
import rasterio as rio
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import tifffile
from affine import Affine
import os
from rasterio.windows import Window
import rasterio
import geopandas as gpd
from shapely.geometry import LineString, Polygon
import glob
from tqdm import tqdm
import detectree as dtr
import pickle
import pandas as pd
import subprocess
from  rasterio.features import geometry_mask
from shapely.geometry import mapping, Polygon
from rasterio.features import shapes
from shapely.geometry import shape, Polygon, MultiPolygon
import warnings

import json

In [6]:
with open('config.json', 'r') as f:
    c = json.load(f)

**Data Preparation**

**Reduction aerial imagery resolution:** 

 
Certainly, the level of detail and accuracy in aerial imagery is directly linked to its spatial resolution. A resolution of 0.5 meters per pixel is generally sufficient for tree extraction, but superior results come with higher spatial resolutions. However, higher resolution implies longer training times for tree detection models, creating a trade-off between accuracy and processing speed. Moreover, the processing time is not just determined by resolution but also by the volume of available aerial imagery. More imagery results in longer processing times. To manage this, reducing spatial resolution minimizes the raw data. All the above must be carefully considered based on the specific project requirements. 

In [7]:
def save_rescaled(tif_path,  output_dir, new_resolution = tuple(c['raw_data']['desired_resolution']), current_resolution= tuple(c['raw_data']['current_resolution'])):

    """
    Rescale a GeoTIFF image while preserving georeferencing and save it as a new GeoTIFF.

    Parameters:
    - tif_path (str): The path to the input GeoTIFF file.
    - output_dir (str): The directory where the rescaled GeoTIFF file will be saved.
    - new_resolution (tuple, optional): The desired new resolution as a tuple of (y_resolution, x_resolution) in units per pixel. Default is (0.45, 0.45).
    - current_resolution (tuple, optional): The current resolution of the input GeoTIFF as a tuple of (y_resolution, x_resolution) in units per pixel. Default is (0.15, 0.15).
    
    Returns:
    - output_tiff (str): The path to the saved rescaled GeoTIFF.
    """

    # Calculate the ratio of the old and new resolutions along X and Y axes
    resolution_ratio_x = current_resolution[1] / new_resolution[1]
    resolution_ratio_y = current_resolution[0] / new_resolution[0]
    
    # Build the output file path for the rescaled GeoTIFF
    output_tiff = os.path.join(output_dir, os.path.basename(tif_path)[:-4] + '_rescaled.tif')

    # Open the original GeoTIFF file using rasterio and obtain metadata
    with rio.open(tif_path) as src:
        # Read image data and metadata
        transform = src.transform
        crs = src.crs

        # Create an Affine transformation matrix with the updated resolution
        new_transform = Affine(transform.a / resolution_ratio_x, transform.b, transform.c,
                        transform.d, transform.e / resolution_ratio_y, transform.f)

    # Read the image using OpenCV
    img = cv2.imread(tif_path)

    # Convert the image from BGR to RGB format
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Calculate new dimensions based on resolution ratio
    new_width = int(img.shape[1] * resolution_ratio_x)
    new_height = int(img.shape[0] * resolution_ratio_y)

    # Perform the image rescaling using bilinear interpolation
    resized = cv2.resize(img_rgb, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Define a new profile for the rescaled GeoTIFF
    profile = {
        'driver': 'GTiff',
        'dtype': np.uint8,
        'width': new_width,
        'height': new_height,
        'count': 3,  # Set count to 3 for RGB image
        'crs': crs,
        'transform': new_transform,
    }

    # Transpose the image data to match the expected shape (3, new_height, new_width)
    resized = np.transpose(resized, (2, 0, 1))

    # Create a new GeoTIFF file with the updated georeferencing and rescaled data
    with rio.open(output_tiff, 'w', **profile) as dst:
        dst.write(resized)
            
    # Return the path to the saved rescaled GeoTIFF
    return output_tiff

# Specify the path to the directory containing the original GeoTIFF files
data = c['path']['raw_data']

# Specify the directory where you want to save the new rescaled GeoTIFF files
output_dir = c['path']['raw_data_rescaled']

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate through all TIFF files in the data directory
for tif_image_path in tqdm(glob.glob(data + '\\' + '*.tif')):
    # Call the save_rescaled function to rescale and save each TIFF file
    path = save_rescaled(tif_image_path, output_dir)

  0%|          | 0/25 [00:00<?, ?it/s]

 60%|██████    | 15/25 [03:44<02:51, 17.18s/it]

**To Tiles:**

After reducing the spatial resolution, the resized data needs to be divided into tiles, each with a resolution not exceeding 1000x1000. This is necessary for the Detectree library, which selects 1% of these tiles to effectively represent the entire dataset for labeling and training the model (explained in detail later).  

In [ ]:
def to_tiles(rescled_path, output_dir, tile_dim= tuple(c['tile']['dim'])):  
    """
    Split a rescaled GeoTIFF image into smaller tiles and save them as separate GeoTIFF files.

    Parameters:
    - rescled_path (str): The path to the rescaled GeoTIFF image to be split into tiles.
    - output_dir (str): The directory where the individual tiles will be saved.
    - tile_dim (tuple, optional): The dimensions of each tile in pixels as (width, height). Default is (268, 365).
    """

    # Open the input GeoTIFF file
    with rio.open(rescled_path) as src:
        # Get metadata and profile information
        profile = src.profile
        transform = src.transform

        # Calculate tile dimensions
        tile_height = tile_dim[1]
        tile_width = tile_dim[0]

        # Get the shape (height and width) of the input GeoTIFF using OpenCV
        img_shape = cv2.imread(rescled_path).shape

        # Iterate through rows and columns to create tiles
        for row in range(img_shape[1] // tile_width):
            for col in range(img_shape[0] // tile_height):
                # Calculate the window for the current tile
                window = Window(row * tile_width, col * tile_height, tile_width, tile_height)

                # Read the data for the current tile
                tile_data = src.read(window=window)

                # Calculate the transform for the current tile
                tile_transform = src.window_transform(window)

                # Create a new profile for the current tile
                tile_profile = src.profile.copy()
                tile_profile.update(width=tile_width, height=tile_height, transform=tile_transform)

                # Generate a file path for the current tile
                tile_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(rescled_path))[0]}_{row}_{col}.tif")

                # Write the tile data as a new GeoTIFF with its own georeferencing
                with rio.open(tile_path, 'w', **tile_profile) as dst:
                    dst.write(tile_data)

    #print("Tiles created successfully.")  # Print a message indicating successful tile creation

# Define the directory where the rescaled images are saved
data = output_dir

# Define the directory where you want to save the tiles
output_dir = c['path']['output_dir_for_tiles']

os.makedirs(output_dir, exist_ok=True)
# Iterate through all TIFF files in the 'data' directory
for tif_image_path in tqdm(glob.glob(data + '*.tif')):
    # Call the 'to_tiles' function to split each rescaled image into tiles remember all the tiles must be at the same size. Therefore the Geotiff's resolution in data directory must be devisible by the tile's resulution.
    to_tiles(tif_image_path, output_dir)

**Select tiles likely to have trees** 

Selecting tiles likely to have trees is a vital step. To improve training efficiency and accuracy, it's best to choose tiles where trees are likely to be present. For instance, in the Porto Santo Project, out of 12,000 tiles, 9,000 were sea tiles, showing only the sea and no trees. To differentiate between tiles that can contain trees (‘land tiles’) and sea tiles, vector data from OpenStreetMap, representing the sea around Porto Santo, was used. In different projects, a similar approach might be needed for other types of tiles, such as desert tiles or tiles showing large lakes. 
 
In this step, a fixed script can be used alongside the user's action of creating polygons for areas without trees. Specifically, the user can utilize QGIS, a popular geographic information system software, to create polygons representing regions devoid of trees. By doing so, the software can then distinguish between areas with trees and those without, aiding in the accurate selection of tiles likely to have trees for further analysis and processing. 

In [1]:
# Path to the shapefile containing polygons of non-tree areas (e.g., water bodies)
shapefile_path_of_areas_non_tree_polygons = c['path']['shapefile_of_non_tree_areas']

In [5]:
def is_tile_intersects_with_object(tile_path ,polylines):
    # Open the GeoTIFF image specified by 'tile_path' using 'rasterio'
    with rasterio.open(tile_path) as src:
         # Get the georeferenced transform
        transform = src.transform

        # Get the image width and height
        width = src.width
        height = src.height

        # Calculate the coordinates of the four corners of the image
        bottom_left = transform * (0, height)
        bottom_right = transform * (width, height)
        top_left = transform * (0, 0)
        top_right = transform * (width, 0)

        # Create a polygon from the coordinates of the four corners
        image_polygon = Polygon([bottom_left, bottom_right, top_right, top_left])

        # Iterate through each polyline in the 'polylines' DataFrame
        for row in polylines.itertuples():
            # Get the LineString geometry from the row
            polyline = row[-1]

            # Check if the tile intersects with the polyline
            if polyline.intersects(image_polygon):
                return True
        
        # If there are no intersections with any polyline, return False
        return False
    
def is_tile_within_object(tile_path, polygons):
    # Initialize a counter to keep track of polygon intersections
    counter = 0
    
    # Open the GeoTIFF file specified by 'tile_path' using 'rasterio'
    with rasterio.open(tile_path) as src:
         # Get the georeferenced transform
        transform = src.transform

        # Get the image width and height
        width = src.width
        height = src.height

        # Calculate the coordinates of the four corners of the image
        bottom_left = transform * (0, height)
        bottom_right = transform * (width, height)
        top_left = transform * (0, 0)
        top_right = transform * (width, 0)

        # Create a polygon from the coordinates of the four corners
        image_polygon = Polygon([bottom_left, bottom_right, top_right, top_left])

        # Iterate through each polygon in the 'polygons' DataFrame
        for row in polygons.itertuples():
            # Get the LineString geometry from the row
            polygon = row[-1]

            # Check if the tile is entirely within the polygon
            if image_polygon.within(polygon):
                return True
            # Check if the tile intersects with the polygon
            elif image_polygon.intersects(polygon):
                counter += 1

    # If the counter is greater than 1, there is at least one intersection
    if counter > 1:
        return True
    
    # If there are no intersections, return False
    return False


# List to store tiles within non-tree areas
non_tree_areas_tiles = []

# List to store tiles intersecting both tree and non-tree areas
tiles_between_areas = []

# List to store tiles within tree areas
tree_area_tiles = []

# Load non-tree area polygons from the shapefile
areas_non_tree_polygons = gpd.read_file(shapefile_path_of_areas_non_tree_polygons)

# Iterate through each tile in the specified directory
for tile_path in tqdm(glob.glob(output_dir + '*tif')):
    # Check if the tile is within non-tree areas
    if is_tile_within_object(tile_path, areas_non_tree_polygons):
        non_tree_areas_tiles.append(tile_path)
    # Check if the tile intersects with both tree and non-tree areas
    elif is_tile_intersects_with_object(tile_path, areas_non_tree_polygons):
        tiles_between_areas.append(tile_path)
    # If the tile is outside non-tree areas, consider it within tree areas
    else:
        tree_area_tiles.append(tile_path)


#saving the paths' lists as a pickle files for future use.
with open('tree_area_tiles.pkl', 'wb') as f:
    pickle.dump(tree_area_tiles, f)

with open('tiles_between_areas.pkl', 'wb') as f:
    pickle.dump(tiles_between_areas, f)

with open('non_tree_areas_tiles.pkl', 'wb') as f:
    pickle.dump(non_tree_areas_tiles, f)

100%|██████████| 12350/12350 [09:54<00:00, 20.78it/s]


**Model Training using Detectree Library (Pythonic Library):**

For additional details about the Detectree library, you can visit the following link: https://github.com/martibosch/detectree. This repository provides in-depth information and resources related to the library, including documentation and usage guidelines. 

In [ ]:
# Select the training tiles from the tiled aerial imagery dataset
# Using the TrainingSelector class from the Detectree library
ts = dtr.TrainingSelector(img_filepaths= tree_area_tiles)
# Split the dataset into training and testing sets using the 'cluster-I' method
split_df = ts.train_test_split(method='cluster-I')

**Data Selection for labeling, Data lableing and Model Training:**

The Detectree tree library automatically chooses 1% of the tiles that most accurately represent the entire set of tiles for user labeling. The 1% tiles should be labeled by the user. 


In [13]:
# This line filters and displays the tiles that should be labeled (training set)
split_df[split_df['train'] == True]

,img_filepath,train
113,tiles\PORTO_SANTO_IMG_B3_rescaled_14_17.tif,True
204,tiles\PORTO_SANTO_IMG_B3_rescaled_20_15.tif,True
343,tiles\PORTO_SANTO_IMG_B3_rescaled_7_12.tif,True
805,tiles\PORTO_SANTO_IMG_B4_rescaled_9_6.tif,True
1001,tiles\PORTO_SANTO_IMG_C2_rescaled_16_4.tif,True
1071,tiles\PORTO_SANTO_IMG_C2_rescaled_20_14.tif,True
1094,tiles\PORTO_SANTO_IMG_C2_rescaled_21_18.tif,True
1245,tiles\PORTO_SANTO_IMG_C3_rescaled_10_6.tif,True
1345,tiles\PORTO_SANTO_IMG_C3_rescaled_17_11.tif,True
1409,tiles\PORTO_SANTO_IMG_C3_rescaled_20_6.tif,True


In [14]:
# Directory to store the training tiles
train_tiles_dir = c['path']['train_tiles_dir']

# Create the directory if it doesn't exist
os.makedirs(train_tiles_dir, exist_ok=True)

# Copy training tiles to the specified directory
for tile_path in split_df[split_df['train'] == True]['img_filepath']:
    shutil.copy(tile_path, train_tiles_dir + os.path.basename(tile_path))

root_dir = ''
images_dir = train_tiles_dir
masks_dir = 'masks\\'
os.makedirs(masks_dir, exist_ok= True)

**Note!**

Before continuing with the code you have to make masks of the training tiles (you can use qgis or gimp).

The training tiles are waiting for you in the training tiles directory

Create masks only for the tiles that has tree in them and save them in the masks directory **with the same name as the original tile**. 

The following code will automaticly detect missing masks and generate blank masks (for tiles without trees). 

In [ ]:
def create_masks(root_dir = '', images_dir = c['path']['train_tiles_dir'], masks_dir = c['path']['masks_dir']):
    # List to store matching filenames of tiles and masks
    matching_files = []

    # Iterate through tiles and masks, match filenames, and store in matching_files list
    for tile_path in glob.glob(root_dir + images_dir + '*tif'):
        for mask_path in glob.glob(root_dir + masks_dir + '*tif'):
            if os.path.basename(tile_path) == os.path.basename(mask_path):
                matching_files.append(os.path.basename(tile_path))

    # Process tiles and masks
    for tile_path in glob.glob(root_dir + images_dir + '*tif'):
        # Check if the tile doesn't have a corresponding mask
        if os.path.basename(tile_path) not in matching_files:
            # Create a black mask
            img = np.zeros(cv2.imread(root_dir + images_dir + os.path.basename(tile_path))[:,:,0].shape)
            data = np.dstack((img, img, img)).astype('uint8')
            
            # Save the mask in RGB format
            tifffile.imwrite(
                root_dir + masks_dir + os.path.basename(tile_path),
                data,
                photometric='rgb',
                bitspersample=8,
                tile=(32, 32),
                planarconfig=1
            )
            
            # Preserve geospatial information from the original tile
            with rio.open(tile_path) as src:
                source_transform = src.transform
                source_crs = src.crs
            
            # Open the mask and preserve its geospatial information
            with rio.open(root_dir + masks_dir + os.path.basename(tile_path), 'r+') as dst:
                dst.transform = source_transform
                dst.crs = source_crs
        else:
            # Load the mask and convert it to binary (0 and 255)
            img = cv2.imread(root_dir + masks_dir + os.path.basename(tile_path))[:,:,0]
            img = np.where(img != 0, 255, 0)
            
            # Save the mask in RGB format
            data = np.dstack((img, img, img)).astype('uint8')
            tifffile.imwrite(
                root_dir + masks_dir + os.path.basename(tile_path),
                data,
                photometric='rgb',
                bitspersample=8,
                tile=(32, 32),
                planarconfig=1
            )
            
            # Preserve geospatial information from the original tile
            with rio.open(tile_path) as src:
                source_transform = src.transform
                source_crs = src.crs
            
            # Open the mask and preserve its geospatial information
            with rio.open(root_dir + masks_dir + os.path.basename(tile_path), 'r+') as dst:
                dst.transform = source_transform
                dst.crs = source_crs

create_masks()

**Start this code and grab a beer it is going to take a while...**

the model will be trained and saved for future use

In [ ]:
# train a tree/non-tree pixel classfier
clf = dtr.ClassifierTrainer().train_classifier(
    split_df=split_df, response_img_dir= c['path']['masks_dir'])

#save the model in case of a crash or something or future use
with open('tree_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

**Predict tree and grass locations and shapefile generation for each tile**

you should see the results in the results folder


In [10]:
os.makedirs('results', exist_ok= True)
subprocess.run(['python', 'multiprocess_tree_and_grass.py'])

**Predict tree and grass locations (not necessary), sometimes it might be completely useless for example in the PortoSanto Project**

also, in order to use this part of the algorithm you have to make sure that the items in the following shapefile ('multipolygons') are MultiPolygon type class.
you can learn more here: https://geopandas.org/en/stable/docs/user_guide/data_structures.html

In [ ]:
# Read the shapefile containing multiple polygons (seas)
multipolygons = gpd.read_file(c['path']['shapefile_of_tiny_water_bodies_or_smalll_area_without_trees'])

In [21]:
def make_masks_of_between_areas():    
    # Ignore the NotGeoreferencedWarning from rasterio
    warnings.filterwarnings("ignore", category=rio.errors.NotGeoreferencedWarning)

    # Load a list of tile paths from a pickle file
    with open('tiles_between_areas.pkl', 'rb') as f:
        tiles_between_areas = pickle.load(f)

    # Create a directory to store the coastline masks if it doesn't exist
    directory = 'coustline_tiles_masks\\' 
    os.makedirs(directory, exist_ok=True)

    # Initialize a MultiPolygon to store merged geometries
    multipolygon_s = multipolygons.iloc[0]['geometry']

    # Iterate through rows of the multipolygons GeoDataFrame and merge the geometries
    for row in multipolygons.itertuples():
        geom = row[-1]
        # Merge the geometries into a single MultiPolygon
        multipolygon_s = multipolygon_s.union(geom)

    # Convert the merged MultiPolygon to a GeoJSON-like mapping
    multipolygon_json = mapping(multipolygon_s)

    # Process each tile path
    for path in tqdm(tiles_between_areas):
        with rio.open(path) as src:
            transform = src.transform
            crs = src.crs

            # Create a mask for the tile using the merged MultiPolygon
            mask = np.where(geometry_mask([multipolygon_json], out_shape=src.shape, transform=transform, invert=True) == True, 1, 0)

        # Define the destination name for the mask TIFF file
        dst_name = directory + os.path.splitext(os.path.basename(path))[0] + '.tif'

        # Save the mask as a TIFF file
        tifffile.imwrite(dst_name, mask.astype('uint8') * 255)

        # Open the created TIFF file for read/write and set georeferencing information
        with rio.open(dst_name, mode='r+') as dst:
            dst.transform = transform
            dst.crs = crs

make_masks_of_between_areas()
subprocess.run(['python', 'multiproccess for tiles_between_areas.py'])

**Create one shapefile of tree locations**

In [ ]:
def clean_false_points(current_tile_with_trees ,shapefile_where_trees_cannot_exist):
    # Initialize a boolean mask for false points, initially all True
    true_points_mask = np.ones(len(current_tile_with_trees), dtype=bool)
    
    # Iterate through rows in the shapefile_where_trees_cannot_exist DataFrame
    for row in shapefile_where_trees_cannot_exist.itertuples():
        # Extract the polygon geometry from the row
        polygon = row[-1]
        
        # Iterate through rows in the current_tile_with_trees DataFrame
        for tile_points_row in current_tile_with_trees.itertuples():
            # Extract the point geometry and its row index from the row
            point = tile_points_row[-1]
            point_index = tile_points_row[0]
            
            # Check if the point is within the polygon
            if point.within(polygon):
                # If it is within the polygon, append its row index to the list
                true_points_mask[i] = False
    
    # Return a new DataFrame containing only rows not in the false_points_index
    return current_tile_with_trees.iloc[true_points_mask]

# Read the shapefile containing polygons where trees cannot exist
shapefile_of_water_bodies_or_locations_where_trees_cannot_exist = c['path']['shapefile_of_tiny_water_bodies_or_smalll_area_without_trees']

# Create an empty GeoDataFrame to store the resulting tree points
shapefile_trees = gpd.GeoDataFrame()

# Iterate through folders and files in the 'results' directory
for folder, _, data in tqdm(os.walk('results')):
    if len(data) != 0:
        # Read the GeoDataFrame containing all tree points for the current tile
        all_tile_points = gpd.read_file(f'{glob.glob(folder + "/*trees.shp")[0]}')
        
        # Clean false points from the current tile using the provided function
        true_tile_point = clean_false_points(all_tile_points, shapefile_of_water_bodies_or_locations_where_trees_cannot_exist)
        
        # Concatenate the resulting true tree points to the shapefile_trees
        shapefile_trees = pd.concat([shapefile_trees, true_tile_point])
# Save the final shapefile containing the cleaned tree points
shapefile_trees.to_file(r'trees.shp')


**Create one shapefile of grass locations**


In [5]:
# Initialize an empty GeoDataFrame to store the grass polygons
shapefile_grass = gpd.GeoDataFrame()

# Iterate through folders and files in the 'results' directory
for folder, _, data in tqdm(os.walk('results')):
    if len(data) != 0:
        try:
            # Concatenate the GeoDataFrame from the current folder's 'grass.shp' file
            shapefile_grass = pd.concat([shapefile_grass, gpd.read_file(f'{folder}\\grass.shp')])
        except:
            print (folder)
            
# Save the final shapefile containing the cleaned grass polygons
shapefile_grass.to_file(r'grass.shp')

80it [00:05, 15.89it/s]
